In [1]:
import pandas as pd
import numpy as np
import re
import fitz

In [2]:
def pdf_to_dict(filename, category):
    doc = fitz.open(f'C:/Users/0727G/Desktop/Projects/Уровень английского/{filename}.pdf') # Открыли файл
    pdf_pages = '' # Создали файл с будущим текстом
    for page in doc: # Цикл по страницам
        text = re.sub(r'©.*\n.*\n.*level??\n|.*level[.]', '', page.get_text()) # Удаление заголовков на каждой странице
        pdf_pages += text # Добавление текста к общему
    pdf_pages = re.sub(r'\s+.+[.]\n', '\n', pdf_pages) # замена на перенос части речи после слова
    pdf_pages = re.split(r'A1\n|A2\n|B1\n|B2\n|C1\n]', pdf_pages) # Создание страниц по уровням
    word_dict = {}
    for page, cat in zip(pdf_pages, category):
        words = re.split(r'\n|\xa0', page)
        result = []
        for line in words:
            word = re.split(' |,', line)[0]
            word = re.sub('[^a-z]*', '', word.lower())
            if word != '':
                result.append(word)
        word_dict[cat] = result
        
        
    return word_dict

In [3]:
dict_oxsford_3000 = pdf_to_dict('The_Oxford_3000_by_CEFR_level', ['A1', 'A2', 'B1', 'B2'])

In [4]:
dict_oxsford_5000 = pdf_to_dict('The_Oxford_5000_by_CEFR_level', ['B2', 'C1'])

In [5]:
dict_american_3000 = pdf_to_dict('American_Oxford_3000_by_CEFR_level', ['A1', 'A2', 'B1', 'B2'])

In [6]:
dict_american_5000 = pdf_to_dict('American_Oxford_5000_by_CEFR_level', ['B2', 'C1'])

In [7]:
pdf_to_dict('The_Oxford_5000_by_CEFR_level', ['C1'])

{'C1': ['the']}

In [8]:
Oxford_df = pd.DataFrame(columns=['words', 'level'])

In [9]:
dicts = [dict_oxsford_3000,
         dict_oxsford_5000,
         dict_american_3000,
         dict_american_5000
        ]
word_dict = {}
for dict in dicts:
    for key in dict.keys():
        if key in word_dict:
            word_dict[key] = set(word_dict[key]) | set(dict[key])
        else:
            word_dict[key] = dict[key]

In [10]:
for lvl in word_dict:
    Oxford_df = pd.concat([Oxford_df, pd.DataFrame({
                                                    'words': [_ for _ in word_dict[lvl]],
                                                    'level':[lvl]*len(word_dict[lvl])
                                                    })])
Oxford_df = Oxford_df.reset_index(drop=True)
Oxford_df

,words,level
0,american,A1
1,the,A1
2,april,A2
3,brother,A2
4,very,A2
...,...,...
4614,jet,C1
4615,lyric,C1
4616,corporation,C1
4617,liable,C1


In [11]:
Oxford_df.drop_duplicates(subset=['words'], inplace=True) # Удалим дубликаты

In [12]:
Oxford_df.dropna(inplace=True)

In [13]:
Oxford_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4366 entries, 0 to 4618
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   words   4366 non-null   object
 1   level   4366 non-null   object
dtypes: object(2)
memory usage: 102.3+ KB


In [14]:
Oxford_df['level'].unique()

array(['A1', 'A2', 'B1', 'B2', 'C1'], dtype=object)

In [15]:
Oxford_df.to_csv("Oxford_classification.csv", index=False)